# Scenario Report

To add:
- [ ] Hourly carbon emissions heatmap
- [ ] Total carbon emissions
- [ ] Percent renewable heatmap
- [ ] Annual average state of charge
- [ ] Fixed annual costs 
- [ ] Weighted average cost of power for each hour or TOU period
- [ ] Cost savings from renewable curtailment
- [ ] Total MWh and shape of curtailment, compared to wholesale prices
- [ ] Marginal emisison metrics
- [ ] Grid impact metrics
- [ ] Organize into sections: (portfolio, cost, dispatch, impact metrics)
- [ ] Duals and RC (non-selected generators)
- [ ] Peak times / binding constraints


To update:
- [ ] Excess generation report (shape, cost, composition)

In [ ]:
# Copyright (c) 2021 *****************. All rights reserved.
# Licensed under the Apache License, Version 2.0, which is in the LICENSE file.

#TODO: delete this
%reload_ext autoreload
%autoreload 2

from pathlib import Path
import pandas as pd
import plotly.express as px
import plotly
import plotly.graph_objects as go 
import numpy as np
from switch_model.reporting.report_functions import *

# TODO: remove input folder
input_folder = '../../MODEL_RUNS/test_PCE/inputs/hourly_90'

#get the name of the current directory to specify the scenario name and identify the output directory
scenario_name = str(Path.cwd() / input_folder).split('\\')[-1]
if scenario_name == 'inputs':
    data_dir = Path.cwd() / input_folder / '../outputs/'
    inputs_dir = Path.cwd() / input_folder / '../inputs/'
    scenario_name = 'N/A'
else:
    data_dir = Path.cwd() / input_folder/ f'../../outputs/{scenario_name}/'
    inputs_dir = Path.cwd() / input_folder / f'../../inputs/{scenario_name}/'

#define formatting options/functions for outputs
pd.options.display.float_format = '{:,.2f}'.format

#allow the notebook to display plots in html report
###################################################
plotly.offline.init_notebook_mode()

print(f'Scenario Name: {scenario_name}')

In [ ]:
# load data from csvs
load_balance = pd.read_csv(data_dir / 'load_balance.csv')
gen_cap = pd.read_csv(data_dir / 'gen_cap.csv')
gen_build_predetermined = pd.read_csv(inputs_dir / 'gen_build_predetermined.csv', usecols=['GENERATION_PROJECT'])
generation_projects_info = pd.read_csv(inputs_dir / 'generation_projects_info.csv')
hybrid_pair = hybrid_pair_dict(generation_projects_info)
costs_by_gen = pd.read_csv(data_dir / 'costs_by_gen.csv')
storage_dispatch = pd.read_csv(data_dir / 'storage_dispatch.csv')
dispatch = pd.read_csv(data_dir / 'dispatch.csv')
system_power = pd.read_csv(data_dir / 'system_power.csv')
costs_by_tp = pd.read_csv(data_dir / 'costs_by_tp.csv')
RA_summary = pd.read_csv(data_dir / 'RA_summary.csv')
fixed_costs = pd.read_csv(inputs_dir / 'fixed_costs.csv')
rec_value = pd.read_csv(inputs_dir / 'rec_value.csv').loc[0,'rec_resale_value'].item()
ra_summary = pd.read_csv(data_dir / 'RA_summary.csv')

technology_color_map = {
 'Small Hydro':'Purple',
 'Consumed Small Hydro':'Purple',
 'Excess Small Hydro':'Plum',
 'Onshore Wind':'DeepSkyBlue',
 'Consumed Onshore Wind':'DeepSkyBlue',
 'Excess Onshore Wind':'LightSkyBlue',
 'Offshore Wind':'Navy',
 'Consumed Offshore Wind':'Navy',
 'Excess Offshore Wind':'MediumSlateBlue',
 'Solar PV':'Gold',
 'Hybrid Solar PV':'Gold',
 'Consumed Solar PV':'Gold',
 'Excess Solar PV':'Yellow',
 'CSP':'Orange',
 'Geothermal':'Sienna',
 'Consumed Geothermal':'Sienna',
 'Storage':'Green',
 'Storage Discharge':'Green',
 'Hybrid Storage':'Green',
 'Grid Energy':'Red',
 '(?)':'Black'}

In [ ]:
for val in fixed_costs['cost_name']:
    print(val)

## Portfolio Renewable Percentage

In [ ]:
  
print(f'Time-coincident renewable percentage: {format_percent(hourly_renewable_percentage(load_balance))}')
print(f'Annual volumetric renewable percentage: {format_percent(annual_renewable_percentage(load_balance))}')

## Generator Portfolio
The sunburst chart describes the built portfolio at various levels of detail, which shows how the outer rings relate to the inner rings
- Inner circle: contract status (contracted or additional project)
- Middle ring: technology type (e.g. solar, wind, ...)
- Outer ring: specific project name

For example, individual projects in the outer ring belong to a specific technology type in the middle ring, which can either be part of the existing/contracted portfolio, or the additional portfolio.


In [ ]:
portfolio = generator_portfolio(gen_cap, gen_build_predetermined)

portfolio_sunburst = px.sunburst(portfolio, path=['Status','Technology','generation_project'], values='MW', color='Technology', color_discrete_map=technology_color_map,
 width=1000, height=1000,
 title='Energy Portfolio by Project Name, Technology Type, and Contract Status (MW)')
portfolio_sunburst.update_traces(textinfo='label+value')
portfolio_sunburst.show()

# Generation Cost by Project
This should show contract costs, Nodal costs, and storage revenues

In [ ]:
gen_costs = generator_costs(costs_by_gen, storage_dispatch, hybrid_pair, gen_cap)

generator_costs_melted = gen_costs.drop(columns='Total Cost').melt(id_vars='generation_project', var_name='Cost', value_name='$/MWh')

generator_cost_fig = px.bar(generator_costs_melted, 
                            title='Average Generator Cost per MWh Generated', 
                            x='generation_project', 
                            y='$/MWh', 
                            color='Cost', 
                            color_discrete_map={'Energy Contract Cost': 'Green',
                                                'Capacity Contract Cost': 'Orange',
                                                'Delivery Cost': 'lightblue',
                                                'Pnode Revenue':'Red', 
                                                'Storage Arbitrage Revenue':'Purple'}).update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='black')

generator_cost_fig.add_scatter(x=gen_costs.generation_project, y=gen_costs['Total Cost'], mode='markers+text', text=gen_costs['Total Cost'], textposition='top center', line=dict(color='black', width=1), name='Total Cost')

generator_cost_fig.show()



# Power Content Label


In [ ]:
power_content = power_content_label(load_balance, dispatch, generation_projects_info)

delivered_energy_pie = px.pie(power_content, values='MWh', names='Source', title='Power Content of Delivered Energy (MWh)', color='Source', color_discrete_map=technology_color_map,
width=600, height=600
 )
delivered_energy_pie.update_traces(textinfo='percent+label+value')
delivered_energy_pie.show()

## Delivered Electricity Cost Summary

In [ ]:
hourly_costs = hourly_cost_of_power(system_power, costs_by_tp, RA_summary, gen_cap, storage_dispatch, fixed_costs, rec_value, load_balance)
hourly_cost_plot = build_hourly_cost_plot(hourly_costs, load_balance)
hourly_cost_plot.show()

### Cost Breakdown

In [ ]:
cost_table, resale_table = construct_cost_and_resale_tables(hourly_costs, load_balance, ra_summary)
display(cost_table.set_index('Cost Component'))


### Resale Values

In [ ]:
display(resale_table.set_index('Cost Component'))

## Resource Adequacy Open Position

In [ ]:
build_ra_open_position_plot(ra_summary).show()

# Supply and Demand Balance

In [ ]:
build_dispatch_plot(generation_projects_info, dispatch, storage_dispatch, load_balance, system_power, technology_color_map).show()

In [ ]:
#only keep observations greater than 0
dispatch_by_tech = dispatch_by_tech[dispatch_by_tech['MWh'] > 0]

dispatch_by_tech['timestamp'] = pd.to_datetime(dispatch_by_tech['timestamp'])

px.area(dispatch_by_tech, x='timestamp', y='MWh', color='Technology', color_discrete_map=technology_color_map, labels={'timestamp':'Datetime','Technology':'Key'})


In [ ]:
# Set up data for dispatch timeseries graph
###########################################

dispatch = pd.read_csv(data_dir / 'dispatch.csv', usecols=['timestamp','generation_project','DispatchGen_MW'], parse_dates=['timestamp'], infer_datetime_format=True)

"""

#create a dictionary that matches hybrid storage to generator
hybrid_pair = pd.read_csv('generation_projects_info.csv', usecols=['GENERATION_PROJECT','storage_hybrid_generation_project'])
hybrid_pair = hybrid_pair[hybrid_pair['storage_hybrid_generation_project'] != "."]
hybrid_pair = dict(zip(hybrid_pair.GENERATION_PROJECT, hybrid_pair.storage_hybrid_generation_project))
"""

try:
    storage = pd.read_csv(data_dir / 'storage_dispatch.csv', parse_dates=['timestamp'], infer_datetime_format=True)
    """
    #merge charging data for hybrid storage projects into the dataframe
    hybrid_dispatch = storage.copy()[['generation_project','timestamp','ChargeMW','DispatchMW']]
    hybrid_dispatch.loc[:,'generation_project'] = hybrid_dispatch['generation_project'].replace(hybrid_pair)
    dispatch = dispatch.merge(hybrid_dispatch, how='left', on=['generation_project','timestamp']).fillna(0)

    # subtract out charging energy from generation from hybrid projects to get true generation
    dispatch.loc[:,'DispatchGen_MW'] = dispatch['DispatchGen_MW'] - dispatch['ChargeMW'] + dispatch['DispatchMW']
    #drop the hybrid columns
    dispatch = dispatch.drop(columns=['ChargeMW','DispatchMW'])
    """
    # rename the column
    dispatch = dispatch.rename(columns={'DispatchGen_MW':'DispatchMW'})


    #append storage 
    storage_dispatch = storage[['generation_project','timestamp','DischargeMW']]
    #storage_dispatch = storage_dispatch[~storage_dispatch['generation_project'].str.contains('HYBRID')]
    dispatch = dispatch.append(storage_dispatch)

    # Setup storage charging timeseries data
    ########################################
    storage_charge = storage[['generation_project','timestamp','ChargeMW']]
    #storage_charge = storage_charge[~storage_charge['generation_project'].str.contains('HYBRID')]
    storage_charge = storage_charge.groupby('timestamp').sum().reset_index()

except FileNotFoundError:
    #calculate total generation
    dispatch = dispatch.rename(columns={'DispatchGen_MW':'DispatchMW'})

#add system power data if available
try:
    system_power_use = pd.read_csv(data_dir / 'system_power.csv', usecols=['timestamp','System_Power_MW'], parse_dates=['timestamp'], infer_datetime_format=True)
    system_power_use.loc[:,'generation_project'] = 'SYSTEM_POWER'
    system_power_use = system_power_use.rename(columns={'System_Power_MW':'DispatchMW'})
    dispatch = dispatch.append(system_power_use)

except FileNotFoundError:
    pass

#add technology column
dispatch['Technology'] = [i.split('_')[0] for i in dispatch['generation_project']]

# replace the technology name of the RE portion of a hybrid with just the name of the technology
for tech in list(dispatch['Technology'].unique()):
    if 'HYBRID' in tech and 'STORAGE' not in tech:
        dispatch.loc[dispatch['Technology'] == tech, 'Technology'] = tech.replace('HYBRID','')

# group the data by technology type
dispatch_by_tech = dispatch.groupby(['Technology','timestamp']).sum().reset_index()
#only keep observations greater than 0
dispatch_by_tech = dispatch_by_tech[dispatch_by_tech['DispatchMW'] > 0]
# Setup load timeseries data
############################

load = pd.read_csv(data_dir / 'load_balance.csv', usecols=['timestamp','zone_demand_mw'], parse_dates=['timestamp'], infer_datetime_format=True)

# Create Figure
###############

color_map = {'HYDRO':'Purple',
 'ONWIND':'Blue',
 'OFFWIND':'Navy',
 'PV':'Yellow',
 'CSP':'Orange',
 'GEO':'Sienna',
 'STORAGE':'Green',
 'STORAGEHYBRID':'GreenYellow',
 'SYSTEM':'Red'}

dispatch_fig = px.area(dispatch_by_tech, x='timestamp', y='DispatchMW', color='Technology', color_discrete_map=color_map, labels={'DispatchMW':'MW','timestamp':'Datetime','Technology':'Key'})
dispatch_fig.update_traces(line={'width':0})
#dispatch_fig.update_traces(hoveron="points+fills")
dispatch_fig.layout.template = 'plotly_white'
#hoveron = 'points+fills'

try:
    # add charging line
    dispatch_fig.add_scatter(x=storage_charge.timestamp, y=(storage_charge.ChargeMW + load.zone_demand_mw), line=dict(color='green', width=4), name='Load + Storage Charge')
except NameError:
    pass

# add load line
dispatch_fig.add_scatter(x=load.timestamp, y=load.zone_demand_mw, line=dict(color='black', width=4), name='Demand')

dispatch_fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1d", step="day", stepmode="backward"),
            dict(count=7, label="1w", step="day", stepmode="backward"),
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(step="all")
        ])))

dispatch_fig.show()

In [ ]:
# Nodal Energy Cost data
nodal_prices = pd.read_csv(inputs_dir / 'nodal_prices.csv')
timestamps = pd.read_csv(inputs_dir / 'timepoints.csv', parse_dates=['timestamp'], usecols=['timepoint_id','timestamp'])
# merge the timestamp data
nodal_prices = nodal_prices.merge(timestamps, how='left', left_on='timepoint', right_on='timepoint_id')

nodal_fig = px.line(nodal_prices, x='timestamp', y='nodal_price', color='pricing_node', labels={'nodal_price':'$/MWh','timestamp':'Datetime','pricing_node':'Node'}, title='Nodal Prices', template='plotly_white')
nodal_fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1d", step="day", stepmode="backward"),
            dict(count=7, label="1w", step="day", stepmode="backward"),
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(step="all")
        ])))
nodal_fig.show()

In [ ]:
# Battery State of Charge data
##############################

try:

    storage = pd.read_csv(data_dir / 'storage_dispatch.csv', parse_dates=['timestamp'], infer_datetime_format=True)
    
    soc = storage.pivot(index='timestamp', columns='generation_project', values='StateOfCharge')
    #remove columns where all values are 0
    soc = soc.loc[:, (soc != 0).any(axis=0)]
    
    #load storage capacity
    storage_energy_capacity = pd.read_csv(data_dir / 'storage_builds.csv', usecols=['generation_project','OnlineEnergyCapacityMWh'], index_col='generation_project')

    #create another dictionary of storage energy capacity summed by storage type
    storage_energy_capacity = storage_energy_capacity.reset_index()
    storage_energy_capacity['generation_project'] = [i.split('_')[0] for i in storage_energy_capacity['generation_project']]
    storage_energy_capacity = storage_energy_capacity.groupby('generation_project').sum()
    grouped_storage_energy_capacity_dict = storage_energy_capacity.to_dict()['OnlineEnergyCapacityMWh']


    #sum by storage type
    soc.columns = [i.split('_')[0] for i in soc.columns]
    
    soc = soc.groupby(soc.columns, axis=1).sum()
    
    #divide by the total capacity to get state of charge
    soc = soc.div(soc.assign(**grouped_storage_energy_capacity_dict))
    
    #soc.index = pd.to_datetime(soc.index)
    
    soc.head(5)

    soc_fig = px.line(soc, x=soc.index, y=list(soc.columns), color_discrete_map={'STORAGE':'green','STORAGEHYBRID':'yellowgreen'}, labels={'timestamp':'Datetime','value':'%'}, title='Storage State of Charge')

    soc_fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1d", step="day", stepmode="backward"),
            dict(count=7, label="1w", step="day", stepmode="backward"),
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(step="all")
        ])))

    soc_fig.show()
    
except FileNotFoundError:
    pass

In [ ]:
mh_dispatch = dispatch.copy()
mh_dispatch = mh_dispatch.set_index('timestamp')

#groupby month and hour
mh_dispatch = mh_dispatch.groupby(['generation_project', mh_dispatch.index.month, mh_dispatch.index.hour], axis=0).mean()
mh_dispatch.index = mh_dispatch.index.rename(['Project','Month','Hour'])
mh_dispatch = mh_dispatch.reset_index()

#add a technology column
mh_dispatch['Technology'] = mh_dispatch['Project'].str.split('_', expand=True)[0]

mh_dispatch = mh_dispatch[mh_dispatch['DispatchMW'] > 0]


#load data
mh_load = pd.read_csv(data_dir / 'load_balance.csv', usecols=['timestamp','zone_demand_mw'], index_col='timestamp', parse_dates=True, infer_datetime_format=True).rename(columns={'zone_demand_mw':'DEMAND'})
mh_load = mh_load.groupby([mh_load.index.month, mh_load.index.hour], axis=0).mean()
mh_load.index = mh_load.index.rename(['Month','Hour'])
mh_load = mh_load.reset_index()

try:
    mh_charge = storage_charge.copy().set_index('timestamp')
    mh_charge = mh_charge.groupby([mh_charge.index.month, mh_charge.index.hour], axis=0).mean()
    mh_charge.index = mh_charge.index.rename(['Month','Hour'])
    mh_charge = mh_charge.reset_index()
    #merge load data
    mh_charge = mh_charge.merge(mh_load, how='left', on=['Month','Hour'])
    mh_charge['ChargeMW'] = mh_charge['ChargeMW'] + mh_charge['DEMAND']
except NameError:
    pass


# Generate the Figure
#####################

color_map = {'HYDRO':'Purple',
 'ONWIND':'Blue',
 'OFFWIND':'Navy',
 'PV':'Yellow',
 'PVHYBRID':'GreenYellow',
 'CSP':'Orange',
 'GEO':'Sienna',
 'STORAGE':'Green',
 'STORAGEHYBRID':'GreenYellow',
 'SYSTEM':'Red'}

mh_fig = px.area(mh_dispatch, x='Hour', y='DispatchMW', facet_col='Month', color='Technology', line_group='Project', color_discrete_map=color_map, facet_col_wrap=6, width=1000, height=600, title='Month-Hour Average Generation Profiles')
mh_fig.layout.template = 'plotly_white'
mh_fig.update_traces(line={'dash':'dot'})

try:
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 1, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 1, 'ChargeMW'], line=dict(color='green', width=4), row=2, col=1, name='STORAGE_Charge', showlegend=True)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 2, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 2, 'ChargeMW'], line=dict(color='green', width=4), row=2, col=2, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 3, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 3, 'ChargeMW'], line=dict(color='green', width=4), row=2, col=3, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 4, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 4, 'ChargeMW'], line=dict(color='green', width=4), row=2, col=4, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 5, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 5, 'ChargeMW'], line=dict(color='green', width=4), row=2, col=5, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 6, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 6, 'ChargeMW'], line=dict(color='green', width=4), row=2, col=6, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 7, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 7, 'ChargeMW'], line=dict(color='green', width=4), row=1, col=1, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 8, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 8, 'ChargeMW'], line=dict(color='green', width=4), row=1, col=2, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 9, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 9, 'ChargeMW'], line=dict(color='green', width=4), row=1, col=3, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 10, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 10, 'ChargeMW'], line=dict(color='green', width=4), row=1, col=4, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 11, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 11, 'ChargeMW'], line=dict(color='green', width=4), row=1, col=5, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 12, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 12, 'ChargeMW'], line=dict(color='green', width=4), row=1, col=6, name='STORAGE_Charge', showlegend=False)
except (NameError, KeyError) as e:
    pass

mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 1, 'Hour'], y=mh_load.loc[mh_load['Month'] == 1, 'DEMAND'], line=dict(color='black', width=4), row=2, col=1, name='Demand', showlegend=True)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 2, 'Hour'], y=mh_load.loc[mh_load['Month'] == 2, 'DEMAND'], line=dict(color='black', width=4), row=2, col=2, name='Demand',showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 3, 'Hour'], y=mh_load.loc[mh_load['Month'] == 3, 'DEMAND'], line=dict(color='black', width=4), row=2, col=3, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 4, 'Hour'], y=mh_load.loc[mh_load['Month'] == 4, 'DEMAND'], line=dict(color='black', width=4), row=2, col=4, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 5, 'Hour'], y=mh_load.loc[mh_load['Month'] == 5, 'DEMAND'], line=dict(color='black', width=4), row=2, col=5, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 6, 'Hour'], y=mh_load.loc[mh_load['Month'] == 6, 'DEMAND'], line=dict(color='black', width=4), row=2, col=6, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 7, 'Hour'], y=mh_load.loc[mh_load['Month'] == 7, 'DEMAND'], line=dict(color='black', width=4), row=1, col=1, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 8, 'Hour'], y=mh_load.loc[mh_load['Month'] == 8, 'DEMAND'], line=dict(color='black', width=4), row=1, col=2, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 9, 'Hour'], y=mh_load.loc[mh_load['Month'] == 9, 'DEMAND'], line=dict(color='black', width=4), row=1, col=3, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 10, 'Hour'], y=mh_load.loc[mh_load['Month'] == 10, 'DEMAND'], line=dict(color='black', width=4), row=1, col=4, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 11, 'Hour'], y=mh_load.loc[mh_load['Month'] == 11, 'DEMAND'], line=dict(color='black', width=4), row=1, col=5, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 12, 'Hour'], y=mh_load.loc[mh_load['Month'] == 12, 'DEMAND'], line=dict(color='black', width=4), row=1, col=6, name='Demand', showlegend=False)

month_names = ['July', 'August', 'September', 'October', 'November', 'December','January', 'February', 'March', 'April', 'May', 'June']
for i, a in enumerate(mh_fig.layout.annotations):
    a.text = month_names[i]


mh_fig.update_xaxes(dtick=3)
mh_fig.show()

In [ ]:
# get month-hour shape of open energy position
##############################################

dispatch_gen = pd.read_csv(data_dir / 'dispatch.csv', usecols=['timestamp','DispatchGen_MW'], parse_dates=['timestamp'], infer_datetime_format=True)

#sum by timestamp
dispatch_gen = dispatch_gen.groupby('timestamp').sum().reset_index()

#sum to get total generation
dispatch_gen['Generation_MW'] = dispatch_gen['DispatchGen_MW']

#merge generation_load_mismatch data
generation_load_mismatch = pd.read_csv(data_dir / 'load_balance.csv', usecols=['timestamp','zone_demand_mw'], parse_dates=['timestamp'], infer_datetime_format=True)

generation_load_mismatch = generation_load_mismatch.merge(dispatch_gen[['timestamp','Generation_MW']], how='left', on='timestamp')

#subtract generation from generation_load_mismatch to get open position and replace negative values with zero
generation_load_mismatch['Overgeneration_MW'] = generation_load_mismatch['Generation_MW'] - generation_load_mismatch['zone_demand_mw']
#generation_load_mismatch.loc[generation_load_mismatch['generation_load_mismatch_MW'] < 0, 'generation_load_mismatch_MW'] = 0

#generation_load_mismatch['timestamp'] = pd.to_datetime(generation_load_mismatch['timestamp'])

generation_load_mismatch = generation_load_mismatch.set_index('timestamp')

generation_load_mismatch_mh = generation_load_mismatch.groupby([generation_load_mismatch.index.month, generation_load_mismatch.index.hour], axis=0).mean()
generation_load_mismatch_mh.index = generation_load_mismatch_mh.index.rename(['Month','Hour'])
generation_load_mismatch_mh = generation_load_mismatch_mh.reset_index()

mh_mismatch_fig = px.area(generation_load_mismatch_mh, x='Hour', y='Overgeneration_MW', facet_col='Month', facet_col_wrap=6, width=1000, height=600, title='Shape of over- and under-generation, excluding energy storage')
mh_mismatch_fig.update_xaxes(dtick=3)
month_names = ['July', 'August', 'September', 'October', 'November', 'December','January', 'February', 'March', 'April', 'May', 'June']
for i, a in enumerate(mh_mismatch_fig.layout.annotations):
    a.text = month_names[i]

mh_mismatch_fig.show()



## Battery Cycles

In [ ]:
try:
    cycles = pd.read_csv(data_dir / 'storage_cycle_count.csv', usecols=['generation_project','storage_max_annual_cycles','Battery_Cycle_Count'])
    cycles = cycles.round(decimals=2)
    cycles = cycles[cycles['Battery_Cycle_Count'] > 0]
    storage_energy_capacity = pd.read_csv(data_dir / 'storage_builds.csv', usecols=['generation_project','OnlineEnergyCapacityMWh'])
    cycles = cycles.merge(storage_energy_capacity, how='left', on='generation_project')
    cycles['Battery_Cycle_Count'] = cycles['Battery_Cycle_Count'] / cycles['OnlineEnergyCapacityMWh']
    cycles = cycles.drop(columns=['OnlineEnergyCapacityMWh']).set_index('generation_project').round(decimals=0)
    cycles['Battery_Cycle_Count'] = cycles['Battery_Cycle_Count'].astype(int)
except FileNotFoundError:
    cycles = "No batteries in model"
cycles

# Generator Assumptions

In [ ]:
pd.set_option('display.max_rows',100)
gen_assumptions = pd.read_csv('generation_projects_info.csv', usecols=['GENERATION_PROJECT','gen_tech','gen_energy_source','gen_reliability_area','ppa_energy_cost','gen_capacity_limit_mw'])
gen_assumptions = gen_assumptions[gen_assumptions['gen_energy_source'] != 'Electricity']
gen_assumptions = gen_assumptions.sort_values(by='GENERATION_PROJECT')
gen_assumptions = gen_assumptions.set_index('GENERATION_PROJECT')
gen_assumptions

# Storage Assumptions

In [ ]:
storage_assumptions = pd.read_csv('generation_projects_info.csv', usecols=['GENERATION_PROJECT','gen_tech','gen_energy_source','gen_reliability_area','ppa_capacity_cost','gen_capacity_limit_mw', 'storage_roundtrip_efficiency','storage_charge_to_discharge_ratio','storage_energy_to_power_ratio','storage_leakage_loss','storage_hybrid_generation_project','storage_hybrid_capacity_ratio'])
#change capacity cost to $/kw-mo
storage_assumptions['ppa_capacity_cost'] = storage_assumptions['ppa_capacity_cost'] / 12000

storage_assumptions = storage_assumptions.rename(columns={
    'storage_roundtrip_efficiency':'RTE',
    'storage_charge_to_discharge_ratio':'charge/discharge_ratio',
    'storage_energy_to_power_ratio':'storage_hours',
    'storage_leakage_loss':'soc_leakage_loss',
    'storage_hybrid_generation_project':'paired_hybrid_gen',
    'storage_hybrid_capacity_ratio':'hybrid_capacity_ratio'
})

storage_assumptions = storage_assumptions[storage_assumptions['gen_energy_source'] == 'Electricity']
storage_assumptions = storage_assumptions.sort_values(by='GENERATION_PROJECT')
storage_assumptions = storage_assumptions.set_index('GENERATION_PROJECT')
storage_assumptions = storage_assumptions[['gen_tech','gen_reliability_area','ppa_capacity_cost','gen_capacity_limit_mw', 'RTE','storage_hours','charge/discharge_ratio','soc_leakage_loss','paired_hybrid_gen','hybrid_capacity_ratio']]
storage_assumptions

In [ ]:
summary = pd.DataFrame(columns=['Scenario Name'], data=['test'])

summary['Scenario Name'] = scenario_name

#Goal Data
summary['Time-coincident Delivered %'] = tc_percent_renewable
summary['Time-coincident Generation %'] = tc_no_storage_percent_renewable
summary['Annual Volumetric Renewable %'] = annual_percent_renewable

unformatted_cost = unformatted_cost.rename(columns={'Annual Real Cost': ' (Annual)',	'Delivered Cost per MWh': ' (per MWh)'}).melt(id_vars=['Cost Component'], var_name='type', value_name=0)
unformatted_cost['col_name'] = unformatted_cost['Cost Component'] + unformatted_cost['type']


summary = pd.concat([summary, unformatted_cost[['col_name',0]].set_index('col_name').T], axis=1)

#Portfolio Mix
portfolio_summary = portfolio[['MW','Status','Technology']].groupby(['Status','Technology']).sum().reset_index()
portfolio_summary['Description'] = portfolio_summary['Status'] + " " + portfolio_summary['Technology']
portfolio_summary = portfolio_summary.drop(columns=['Status','Technology'])
portfolio_summary = portfolio_summary.set_index('Description').transpose().reset_index(drop=True).add_prefix('MW Capacity from ')

summary = pd.concat([summary, portfolio_summary], axis=1)

#Load
summary['Customer Load GWh'] = load['zone_demand_mw'].sum(axis=0) / 1000
try:
    summary['Total Load with Storage GWh'] = (load['zone_demand_mw'].sum(axis=0) + storage_charge['ChargeMW'].sum(axis=0)) / 1000
except (KeyError, NameError):
    summary['Total Load with Storage GWh'] = summary['Customer Load GWh']

#Generation Mix
generation_summary = generation_mix.set_index('Source').transpose().reset_index(drop=True).add_prefix('GWh Generation from ')
summary = pd.concat([summary, generation_summary], axis=1)

summary = summary.transpose()
summary.columns = [f'{scenario_name}']

summary.to_csv(data_dir / 'scenario_summary.csv')